In [2]:
import requests                  
import pandas as pd              
import numpy as np               
import matplotlib.pyplot as plt  
import time            
        
from tqdm import tqdm_notebook   
import vk
import pickle

In [3]:
access_token = '...'
my_id = '...'

session = vk.Session(access_token=access_token)
my_api = vk.API(session, v='5.21')

In [5]:
user_info = my_api.users.get(user_ids=my_id)
print(user_info)
start_id = user_info[0]['id']

[{'id': 567890010, 'first_name': 'Dmitry', 'last_name': 'Pimenov'}]


In [6]:
infa = my_name['response'][0]
infa

{'id': 567890010, 'first_name': 'Dmitry', 'last_name': 'Pimenov'}

In [18]:
my_api.wall.get(owner_id = my_id)['items']

[{'id': 3266,
  'from_id': 567890010,
  'owner_id': 567890010,
  'date': 1583390745,
  'post_type': 'post',
  'text': 'Я принял участие в голосовании «Благоустройство парка Северного речного вокзала      » в проекте «Активный гражданин» и получил 20 баллов',
  'attachments': [{'type': 'link',
    'link': {'url': 'https://ag.mos.ru/poll/7201?ref=-Y_d5iO5ch',
     'title': 'Активный Гражданин - проект для тех, кому важно, что происходит в Москве',
     'description': 'Ты решаешь! Электронные голосования правительства Москвы.',
     'target': 'internal',
     'image_src': 'https://sun9-5.userapi.com/c855124/v855124509/203a7c/sAWFQg9-N9E.jpg'}}],
  'comments': {'count': 0},
  'likes': {'count': 1},
  'reposts': {'count': 1}},
 {'id': 3265,
  'from_id': 567890010,
  'owner_id': 567890010,
  'date': 1583390720,
  'post_type': 'post',
  'text': 'Я принял участие в голосовании «Благоустройство прибрежной части реки Чермянки» в проекте «Активный гражданин» и получил 20 баллов',
  'attachments':

In [17]:
[item['likes']['count'] for item in my_api.wall.get(owner_id = my_id)['items']]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

Ура, оно работает, попытаемся сделать что-то полезное

Примерный план полезного у нас такой:

Давай начнём с какого нибудь вк. Действовать будем таким путём - обозначим какую нибудь тему за которой мы хотим мониторить - нууу например обучение (можно что угодно ещё)

Возьмём за основу какие нибудь группы популярные где такое есть - ну гик брейнс, можно шад, что нибудь такое. Будем парсить их посты через апишку и подписчиков. У подписьючиков будем искать ещё группы и смотреть потенциальные места и рассчирять список групп, у постов будем смотреть охватность и искать какие нибудь маркерные слова что это рекламный пост (ну или явную запись что пост рекламный), сверять охватности

Вот собственно план на начало)

Потом между постами будем искать модельки близости


https://vk.com/geekbrainsru - первая цель. Украдём посты и подписичников

In [69]:
id_geek = '15365973'
id_volley = '65570928'

In [22]:
my_api.wall.get(owner_id=id_geek, count = 5)['items']

[{'id': 1296470,
  'from_id': -15365973,
  'owner_id': -15365973,
  'date': 1599493080,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': 'Вместо мишени — данные. Вместо винтовки — командная строка. Вместо шума болельщиков — тихий, размеренный гул процессорного вентилятора. Вместо проторенной лыжни разум мчится к цели новыми, неизведанными тропами, сквозь файрволы, аутентификации и все возможные скрипты. Да, это не хуже какого-нибудь биатлона — соревнования Capture the Flag для знатоков инфобезопасности.\n\nВ третьем выпуске подкаста мы поговорим о таких вот играх разума с тем, кто организует их в GeekBrains — Виктором Чаплыгиным, администратором CTF-стенда для студентов-кибербезопасников в GeekUniversity и GeekSchool. Да, и GeekSchool в том числе — это именно тот случай, когда школьные олимпиады могут выйти из класса 😉',
  'attachments': [{'type': 'podcast',
    'podcast': {'artist': 'GeekBrains',
     'id': 456239069,
     'owner_id': -15365973,
     'title': 'Эпизод 3. CTF (Cap

In [23]:
#крадем лайки с первых 5 постов, чтобы убедиться, что всё работает

[item['likes']['count'] for item in my_api.wall.get(owner_id=id_geek, count = 5)['items']]

[39, 40, 12, 17, 67]

In [76]:
my_api.wall.get(owner_id = '-'+id_geek)['count']

5436

In [85]:
def pars_group(group_id): 
    # В этот словарик будем собирать информацию о текстах, можно напихать что-нибудь еще
    wall_inf = {'text' : [ ],
               'id' : [ ]}

    # Выясним сколько всего в группе постов 
    n = my_api.wall.get(owner_id = '-' + group_id)['count']
    print('В группе ', n, 'постов')

    for i in tqdm_notebook(range(0, n + 100, 100)): 
        # После каждого запроса будем ждать 0.4 секунды, иначе вк не отдаст данные. 
        time.sleep(0.4)
        wall = my_api.wall.get(owner_id = '-' + group_id, count = 100, offset = str(i))['items']
        # Вытащили тексты постов и id
        new_text = [item['text'] for item in wall]
        post_ids = [item['id'] for item in wall]
        # Закинули всё это добро в словарик с данными 
        wall_inf['text'].extend(new_text)
        wall_inf['id'].extend(post_ids)
    return wall_inf


In [87]:
pars_wall = pars_group(id_volley)

В группе  530 постов


In [88]:
df_post = pd.DataFrame(pars_wall)
df_post.head()

,text,id
0,Здравствуйте. А в 27 школееще игры собираются?...,2905
1,Приглашаем любителей спорта на игры:\n- ВОЛЕЙБ...,2904
2,Прекрасные летние игры! 😍😍😍\n#волейбол #мытищи...,2900
3,"Всем привет ,а на торпедо как раньше не собира...",2896
4,"Пришло лето, пора играть в волейбол на улице. ...",2891


In [132]:
df_copy = df_post.copy()

Отлично, мы умеем получать текст и id постов из рандомных групп. Теперь поучимся считать охват.

In [490]:
def pars_comment(pars_wall, group_id):    
    comments_id = []
    commentator_id = {}

    for post_id in tqdm_notebook(pars_wall['id']):
        info = my_api.wall.getComments(owner_id= '-' + group_id, post_id=post_id)['items']
        topic = [
            {            
                'comment_id': item['id'],
                'post_id' : post_id
            } for item in info
        ]
        comments_id.extend(topic)

        time.sleep(0.4)
        
        dict_id_commenst = [
            {
                'user_id': item['from_id']
            } for item in info
        ]

        commentator_id[post_id] = dict_id_commenst
        
        
        
    print('В группе ', len(comments), 'комментариев')
    
    df_post_user = pd.DataFrame([(comment) for comment in commentator_id.items()], 
                                columns=['id', 'user_id_comment'])
    df_post_user['user_id_comment'] = df_post_user['user_id_comment'].apply(lambda x: [id['user_id'] for id in x])
    df_post_user.set_index('id', inplace=True)
    
    return df_post_user, comments_id
    

In [446]:
df_post_user_volley, comments_id_volley = pars_comment(pars_wall)


В группе  14 комментариев


In [401]:
df_post_user_volley

,user_id_comment
id,
2905,[]
2904,[]
2900,"[24528299, 16714, 41817148, 155411153, 16714, ..."
2896,"[5595762, 151064528]"
2891,"[607300, 1023502, 5595762, 6739971, 2633469]"
...,...
63,"[14869434, 251396370, 136649449, 231001361, 71..."
41,"[14869434, 136649449, 14869434, 136649449, 251..."
13,"[14869434, 14869434, 5843441, 14869434, 251396..."


In [346]:
df_post = df_post.join(df_post_user_volley, on = 'id')

In [342]:
def pars_likes_users_post(pars_wall, group_id):    
    likes_users_post = []
    likes_id = {}
    for post_id in tqdm_notebook(pars_wall['id']):
        likes_id[post_id] = my_api.likes.getList(type='post', 
                                                      owner_id='-' + group_id,
                                                      item_id=post_id)['items']

        time.sleep(0.4)
    
    df_likes_user = pd.DataFrame([(like) for like in likes_id.items()], columns=['id', 'user_id_like'])
    df_likes_user.set_index('id', inplace=True)
    
    return df_likes_user

In [343]:
df_like_user_volley = pars_likes_users_post(pars_wall)

In [403]:
# df_post = df_post.join(df_like_user_volley, on = 'id')
df_post.head()

,text,id,user_id_comment,user_id_like
0,Здравствуйте. А в 27 школееще игры собираются?...,2905,[],[]
1,Приглашаем любителей спорта на игры:\n- ВОЛЕЙБ...,2904,[],[]
2,Прекрасные летние игры! 😍😍😍\n#волейбол #мытищи...,2900,"[24528299, 16714, 41817148, 155411153, 16714, ...","[607300, 2633469, 2957096, 3039983, 5595762, 1..."
3,"Всем привет ,а на торпедо как раньше не собира...",2896,"[5595762, 151064528]",[]
4,"Пришло лето, пора играть в волейбол на улице. ...",2891,"[607300, 1023502, 5595762, 6739971, 2633469]","[607300, 1023502, 2085261, 110386169, 240643150]"


In [493]:
def pars_likes_post_comment(comments_id, group_id):

    likes_users_comment = []
    for comments_id, post_id in tqdm_notebook(zip([item['comment_id'] for item in comments_id],
                                                 [item['post_id'] for item in comments_id])):
        info = my_api.likes.getList(type='comment', 
                                    owner_id='-' + group_id, 
                                    item_id=comments_id)['items']

        topic = [
                {            
                    'like_comment_id': item,
                    'post_id' : post_id
                } for item in info
            ]
        likes_users_comment.extend(topic)
        time.sleep(0.34)

    # здесь начинается кусок кода с болью, понять как это работает и как я к этому пришел, наверное,
    # будет сложно, но очень хотелось сделать красиво, менее рукожопного способа я не нашел


    df_tmp = pd.DataFrame(likes_users_comment).set_index('post_id') # пока все понятно
    df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: str(x) + ' ') # делем строки из id
    df_tmp = pd.DataFrame(df_tmp.groupby('post_id')['like_comment_id'].sum()) # группируем лайки по id поста
    df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: (x.split(' '))) # делаем красиво
    df_tmp['like_comment_id'] = df_tmp['like_comment_id'].apply(lambda x: x[:-1])

    return df_tmp
    

In [495]:
df_volley_comm_like = pars_likes_post_comment(comments_id_volley, id_volley)

In [496]:
df_post = df_post.join(df_volley_comm_like, on = 'id')

In [497]:
df_post

,text,id,user_id_comment,user_id_like,like_comment_id
0,Здравствуйте. А в 27 школееще игры собираются?...,2905,[],[],NaN
1,Приглашаем любителей спорта на игры:\n- ВОЛЕЙБ...,2904,[],[],NaN
2,Прекрасные летние игры! 😍😍😍\n#волейбол #мытищи...,2900,"[24528299, 16714, 41817148, 155411153, 16714, ...","[607300, 2633469, 2957096, 3039983, 5595762, 1...",NaN
3,"Всем привет ,а на торпедо как раньше не собира...",2896,"[5595762, 151064528]",[],NaN
4,"Пришло лето, пора играть в волейбол на улице. ...",2891,"[607300, 1023502, 5595762, 6739971, 2633469]","[607300, 1023502, 2085261, 110386169, 240643150]",[5595762]
...,...,...,...,...,...
525,"09.05.2014 пят. - время примерно 12-13 часов, ...",63,"[14869434, 251396370, 136649449, 231001361, 71...","[32726921, 136649449, 231001361]",NaN
526,"01.05.2014 четв. - время примерно 12-13 часов,...",41,"[14869434, 136649449, 14869434, 136649449, 251...",[14869434],NaN
527,27.04. вс при наличии хорошей погоды есть пред...,13,"[14869434, 14869434, 5843441, 14869434, 251396...","[5843441, 20177799, 136649449]",NaN
528,"19.04.2014 - 16-17 часов, есть хорошая возможн...",3,"[14869434, 4717972, -65570928, 51872982, 14869...","[14869434, 165189314]",NaN


Теперь у нас есть такая красота. По столбцам у нас:

- пост
- id поста
- id пользователей, оставивших комментарий
- id пользователей, оставивших лайк под постом
- id пользователей, оставивших лайк под комментариями

на основе этого можно уже придумывать какие-то модели, пихать в word2vec, ну и на что фантазии хватит. Хотя фьючей хотелось бы, наверное, побольше)

In [502]:
# Сохраняем накачаное добро 
with open('vk_volley_df', 'wb') as f:
    pickle.dump(df_post, f)

In [498]:
my_api.groups.getMembers(group_id = id_geek)

VkAPIError: 15. Access denied: group hide members. request_params = {'method': 'groups.getMembers', 'oauth': '1', 'v': '5.21', 'group_id': '15365973'}

Злые люди скрыли своих подписичников, зато мы теперь можем написать такую штуку

In [499]:
def pars_group_people(group_id): 
    # В этот словарик будем собирать информацию о текстах, можно напихать что-нибудь еще
    wall_inf = {'members' : [ ]}

    # Выясним сколько всего в группе подписчиков
    try:
        n = my_api.groups.getMembers(group_id = group_id)['count']
    except Exception:
        print('HiddenMembers?')
        wall_inf['members'].append('HiddenMembers')
        
    else:
        
        print('В группе ', n, 'подписчиков')

        for i in tqdm_notebook(range(0, n + 100, 100)): 
            # После каждого запроса будем ждать 0.4 секунды, иначе вк не отдаст данные. 
            time.sleep(0.4)
            people = my_api.groups.getMembers(
                group_id = group_id, 
                count = 100, 
                offset = str(i)
            )['items']
            # Вытащили подписчиков 
            # Закинули всё это добро в словарик с данными 
            wall_inf['members'].extend(people)
    return wall_inf


In [500]:
pars_volley_people = pars_group_people(id_volley)

df_people = pd.DataFrame(pars_volley_people)
df_people.head()

В группе  508 подписчиков


,members
0,90970
1,118372
2,126033
3,277786
4,504582


Из-за ограничения VK по количеству запросов в секунду, все это работает невероятно долго. Именно поэтому я взял маленькую группу по волейболу) но даже в ней стена парсится небыстро. Документация говорит, что есть волшебный метод execute, который позволяет ускорить работу в 25 раз, но как он работает напрямую из API, я чет, если честно, не разобрался ;(